In [1]:
import pickle as pk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [2]:
CLEAN_PATH = "datasets/cleaned_dataset.csv"
VECTORIZER_PATH = "models/vectorizer.pickle"
CLASSIFIER_PATH = "models/classifier.pickle"

In [3]:
clean_dataset = pd.read_csv(CLEAN_PATH)

In [4]:
vectorizer = CountVectorizer(stop_words=stopwords.words("english"))
x = vectorizer.fit_transform(clean_dataset["keywords"])
y = clean_dataset["classification"]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)

In [5]:
classifier = MultinomialNB().fit(x_train, y_train)

In [6]:
y_pred = classifier.predict(x_test)
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

not_offensive       0.75      0.73      0.74       680
    offensive       0.64      0.66      0.65       488

     accuracy                           0.70      1168
    macro avg       0.69      0.70      0.69      1168
 weighted avg       0.70      0.70      0.70      1168



In [7]:
# comment = "welcome to reddit"
# features = vectorizer.transform([comment])
# prediction = classifier.predict_proba(features)
# print(f"offensiveness: {prediction[0][1]}")

In [8]:
with open(VECTORIZER_PATH, "wb") as vectorizer_file,open(CLASSIFIER_PATH, "wb") as classifier_file:
    pk.dump(vectorizer, vectorizer_file)
    pk.dump(classifier, classifier_file)
